# Framing Aggregates with WINDOW Functions

Welcome! This session assumes an intermediate understanding and usage of SQL

WINDOW functions in SQL allows to change the lens of the same data we are querying by adding context to the values being used. By design, window functions help you aggregate numbers or metadata so that each data row also contains general information related to the attributes

## What we're doing today

By the end of today, you will have become familiar with two window functions in SQL that allow you to group and aggregate values

- OVER, PARTITION BY, ORDER BY

- Aggregation of numerical values

### You will have the chance to follow along

- Go to deepnote.com

- Follow this video to create a project

- Go to github.com/siawayforward/training-sessions and go to the folder called sql-training-session-202310 and download the following files: window_functions_session.ipynb and airport_data.csv

- Upload the two files into the Deepnote project you created

- Open the notebook and press Run Notebook on the top right section to load your virtual environment

# Let's Get Into It

## Loading and Previewing Data

Before we do anything to our data, let's see and understand it! By the end of this, we will know what our data table looks like, what each column represents, and what the smallest grain is i.e. what combination of attributes makes a row unique

- We have a CSV file of airport travel for the month of October 2023 in local US airports

- We will make the assumptions that our data rows are unique and the departure and destination airport combination never equals each other i.e. you can't have the same departure and destination airport for a passenger

In [1]:
preview = _deepnote_execute_sql('SELECT *\nFROM \'airport_data.csv\'\nLIMIT 5;', 'SQL_DEEPNOTE_DATAFRAME_SQL', audit_sql_comment='', sql_cache_mode='cache_disabled')
preview

,travel_date,passenger_name,passenger_age,departure_airport,destination_airport,travel_window
0,2023-10-04,Trevor Thompson,21,ORD,ATL,PM
1,2023-10-11,Ashley Martinez,31,ATL,JFK,PM
2,2023-10-02,Maria Harris,9,LAX,OAK,AM - Early
3,2023-10-05,Steven Wood,48,ORD,LAX,PM - Late
4,2023-10-12,Stephanie Meyers,44,DET,ORD,AM


In [2]:
airport_traffic = _deepnote_execute_sql('-- Select data so we can save it as a variable\nSELECT *\nFROM \'airport_data.csv\'\n;', 'SQL_DEEPNOTE_DATAFRAME_SQL', audit_sql_comment='', sql_cache_mode='cache_disabled')
airport_traffic

,travel_date,passenger_name,passenger_age,departure_airport,destination_airport,travel_window
0,2023-10-04,Trevor Thompson,21,ORD,ATL,PM
1,2023-10-11,Ashley Martinez,31,ATL,JFK,PM
2,2023-10-02,Maria Harris,9,LAX,OAK,AM - Early
3,2023-10-05,Steven Wood,48,ORD,LAX,PM - Late
4,2023-10-12,Stephanie Meyers,44,DET,ORD,AM
...,...,...,...,...,...,...
135364,2023-10-01,Debbie Thompson,45,DET,ATL,Red Eye
135365,2023-10-04,Alice Salazar,59,MSP,LGA,AM - Early
135366,2023-10-02,Danielle Russell,6,DCA,SFO,PM - Late
135367,2023-10-02,John Guerrero,42,JFK,ORD,Red Eye


## Now, our functions

- OVER - while selecting a specific column, you can group things using PARTITION BY, and order things using ORDER BY

Let's try it!

ORDER BY - How would we track the volume of passengers changes each day by departure airport?

There are three parts of grouping to consider. We want

- Average age = we need to use the COUNT function

- For each day = means we need to group our count by day

- By departure airport = means we also need a grouping that breaks out departure too

First, the typical grouping usage would look something like this

In [3]:
daily_passenger_departures = _deepnote_execute_sql('SELECT \n    travel_date,\n    departure_airport,\n    -- our aggregation function\n    COUNT(passenger_name) AS passenger_ct\nFROM airport_traffic\nGROUP BY 1, 2\nORDER BY 1, 2', 'SQL_DEEPNOTE_DATAFRAME_SQL', audit_sql_comment='', sql_cache_mode='cache_disabled')
daily_passenger_departures

,travel_date,departure_airport,passenger_ct
0,2023-10-01,ATL,512
1,2023-10-01,DCA,572
2,2023-10-01,DET,598
3,2023-10-01,JFK,554
4,2023-10-01,LAS,587
...,...,...,...
215,2023-10-20,LGA,556
216,2023-10-20,MSP,561
217,2023-10-20,OAK,546
218,2023-10-20,ORD,1029


Now what if we wanted to get the change over time? In the above case, you may create a new column after downloading this. Maybe in Excel with a pivot table. But what if we could get the cumulative number of customers each day with SQL?! Here come our window functions! 

In [4]:
df_3 = _deepnote_execute_sql('WITH cumulative_sum AS (\n    SELECT\n        travel_date,\n        departure_airport,\n        passenger_ct AS passengers_today,\n        -- our aggregation function with grouping by departure\n        SUM(passenger_ct) \n            -- partition by defines the groups of things we want to sum together\n            -- order by indicates the sequence of how we sum things up. Default is ASC ordering\n            OVER(PARTITION BY departure_airport ORDER BY travel_date) AS passenger_running_total\n    FROM daily_passenger_departures\n\n)\nSELECT * \nFROM cumulative_sum\nWHERE departure_airport = \'DET\';', 'SQL_DEEPNOTE_DATAFRAME_SQL', audit_sql_comment='', sql_cache_mode='cache_disabled')
df_3

,travel_date,departure_airport,passengers_today,passenger_running_total
0,2023-10-01,DET,598,598
1,2023-10-02,DET,599,1197
2,2023-10-03,DET,605,1802
3,2023-10-04,DET,559,2361
4,2023-10-05,DET,586,2947
5,2023-10-06,DET,607,3554
6,2023-10-07,DET,549,4103
7,2023-10-08,DET,593,4696
8,2023-10-09,DET,542,5238
9,2023-10-10,DET,570,5808


You could also do a grouping by departure and destinations. Let's combine the two steps into one

In [5]:
running_totals_by_departure = _deepnote_execute_sql('-- first query where we count our groupings by departure and for the day using window function\nSELECT \n    travel_date,\n    passenger_name,\n    passenger_age,\n    departure_airport,\n    destination_airport,\n    -- include the grouping for passenger count by day for the departure\n    COUNT(1) OVER(PARTITION BY travel_date, departure_airport ORDER BY travel_date) AS passenger_departures_ct,\n    -- include the cumulative grouping to increment the passenger count by day\n    SUM(1) OVER(PARTITION BY departure_airport ORDER BY travel_date) AS passenger_departures_running_ct\nFROM airport_traffic\n', 'SQL_DEEPNOTE_DATAFRAME_SQL', audit_sql_comment='', sql_cache_mode='cache_disabled')
running_totals_by_departure

,travel_date,passenger_name,passenger_age,departure_airport,destination_airport,passenger_departures_ct,passenger_departures_running_ct
0,2023-10-01,Mr. Gary Boyd,45,DET,JFK,598,598
1,2023-10-01,Evelyn Lopez,60,DET,OAK,598,598
2,2023-10-01,Mr. William Jenkins,65,DET,JFK,598,598
3,2023-10-01,Tammy Jarvis,13,DET,OAK,598,598
4,2023-10-01,Christopher Higgins,7,DET,LAS,598,598
...,...,...,...,...,...,...,...
135364,2023-10-20,Madison Ramirez,17,ORD,SFO,1029,20985
135365,2023-10-20,Charles Gomez,59,ORD,LAS,1029,20985
135366,2023-10-20,Amy Duran,58,ORD,LAS,1029,20985
135367,2023-10-20,Meghan Stevens,35,ORD,SFO,1029,20985


Notice: you don't have to use a GROUP BY function with the aggregation above because you are not reducing rows, you are adding more information to each row

Now let's select one departure so we can see how the count changes with each day to make sure we got what we wanted

In [6]:
df = _deepnote_execute_sql('-- you can check to see your work for a specific airport\nSELECT \n    -- window functions are not meant to aggregate rows without you don\'t see them\n    -- it is meant for you to get the aggregate within the smallest grain\n    -- distinct in production is discouraged, this is to show an example of one departure locale with the running totals\n    DISTINCT \n    travel_date, departure_airport, passenger_departures_ct, passenger_departures_running_ct\nFROM running_totals_by_departure\nWHERE departure_airport = \'DET\'', 'SQL_DEEPNOTE_DATAFRAME_SQL', audit_sql_comment='', sql_cache_mode='cache_disabled')
df

,travel_date,departure_airport,passenger_departures_ct,passenger_departures_running_ct
0,2023-10-01,DET,598,598
1,2023-10-02,DET,599,1197
2,2023-10-03,DET,605,1802
3,2023-10-04,DET,559,2361
4,2023-10-05,DET,586,2947
5,2023-10-06,DET,607,3554
6,2023-10-07,DET,549,4103
7,2023-10-08,DET,593,4696
8,2023-10-09,DET,542,5238
9,2023-10-10,DET,570,5808


### Let's try this again with the route (departure - destination combination)

In [7]:
running_totals_by_route = _deepnote_execute_sql('-- first query where we count our groupings by departure x destination\nSELECT \n    travel_date,\n    passenger_name,\n    departure_airport,\n    destination_airport,\n    -- include the grouping for passenger count by day for the departure x destination\n    COUNT(1) OVER(PARTITION BY travel_date, departure_airport, destination_airport ORDER BY travel_date) AS passenger_routes_ct,\n    -- include the cumulative grouping to increment the passenger count by day\n    SUM(1) OVER(PARTITION BY departure_airport, destination_airport ORDER BY travel_date) AS passenger_routes_running_ct\nFROM airport_traffic', 'SQL_DEEPNOTE_DATAFRAME_SQL', audit_sql_comment='', sql_cache_mode='cache_disabled')
running_totals_by_route

,travel_date,passenger_name,departure_airport,destination_airport,passenger_routes_ct,passenger_routes_running_ct
0,2023-10-01,James Garcia,DCA,MSP,60,60
1,2023-10-01,Sherry Stevenson,DCA,MSP,60,60
2,2023-10-01,Megan Turner,DCA,MSP,60,60
3,2023-10-01,Shannon Lynch,DCA,MSP,60,60
4,2023-10-01,Kaitlyn Hernandez,DCA,MSP,60,60
...,...,...,...,...,...,...
135364,2023-10-20,Emily Evans,OAK,JFK,52,998
135365,2023-10-20,Kaitlyn May,OAK,JFK,52,998
135366,2023-10-20,Scott Saunders,OAK,JFK,52,998
135367,2023-10-20,Sydney Callahan,OAK,JFK,52,998


In [8]:
df_8 = _deepnote_execute_sql('-- easy check for this\nSELECT \n    DISTINCT\n        travel_date, departure_airport, destination_airport, passenger_routes_ct, passenger_routes_running_ct\nFROM running_totals_by_route\nWHERE \n    departure_airport = \'DCA\' \n    AND destination_airport = \'MSP\'', 'SQL_DEEPNOTE_DATAFRAME_SQL', audit_sql_comment='', sql_cache_mode='cache_disabled')
df_8

,travel_date,departure_airport,destination_airport,passenger_routes_ct,passenger_routes_running_ct
0,2023-10-01,DCA,MSP,60,60
1,2023-10-02,DCA,MSP,48,108
2,2023-10-03,DCA,MSP,49,157
3,2023-10-04,DCA,MSP,50,207
4,2023-10-05,DCA,MSP,58,265
5,2023-10-06,DCA,MSP,51,316
6,2023-10-07,DCA,MSP,59,375
7,2023-10-08,DCA,MSP,63,438
8,2023-10-09,DCA,MSP,46,484
9,2023-10-10,DCA,MSP,45,529


How can we make this more robust?

- Dates: our data only has October, so our running month total doesn't have to check the date

- Naming is important

- You could have a WINDOW defined for each selection so you can create table indices and views

In [9]:
df_9 = _deepnote_execute_sql('-- let\'s account for the date. what if we want the monthly running totals and we have more than October?\nWITH travel_month_dataset AS (\n    -- same query as before; let\'s put this in a CTE\n    -- first query we set up our travel month variable which we will use in the groupings in our window function\n    SELECT \n        travel_date,\n        EXTRACT(YEAR FROM travel_date) || \'-\' || EXTRACT(MONTH FROM travel_date) AS travel_month,\n        passenger_name,\n        departure_airport,\n        destination_airport\n    FROM airport_traffic\n\n), cumulative_route_ct AS (\n    SELECT  \n        *,\n        -- include the grouping for passenger count by day, month, and the departure x destination\n        COUNT(1) OVER(PARTITION BY travel_date, departure_airport, destination_airport ORDER BY travel_date) AS passenger_routes_ct,\n        -- include the cumulative grouping to increment the passenger count by day and month\n        SUM(1) OVER(PARTITION BY departure_airport, destination_airport ORDER BY travel_date) AS passenger_routes_running_ct\n    FROM travel_month_dataset\n    \n)\n-- let\'s check a specific route.\n-- it will look the same as before, but if we had multiple months, the cumulative values would reset each month\nSELECT DISTINCT\n    travel_date, departure_airport, destination_airport, passenger_routes_ct, passenger_routes_running_ct\nFROM cumulative_route_ct\nWHERE \n    departure_airport = \'DCA\' \n    AND destination_airport = \'MSP\'', 'SQL_DEEPNOTE_DATAFRAME_SQL', audit_sql_comment='', sql_cache_mode='cache_disabled')
df_9

,travel_date,departure_airport,destination_airport,passenger_routes_ct,passenger_routes_running_ct
0,2023-10-01,DCA,MSP,60,60
1,2023-10-02,DCA,MSP,48,108
2,2023-10-03,DCA,MSP,49,157
3,2023-10-04,DCA,MSP,50,207
4,2023-10-05,DCA,MSP,58,265
5,2023-10-06,DCA,MSP,51,316
6,2023-10-07,DCA,MSP,59,375
7,2023-10-08,DCA,MSP,63,438
8,2023-10-09,DCA,MSP,46,484
9,2023-10-10,DCA,MSP,45,529


## Further Learning

I hope you have enjoyed learning about this window function. To productionize your data queries, you may use the WINDOW function to define a specific window through which to apply aggregations for your rows. This function works with MySQL databases and in BigQuery. There are other ways to aggregate information for both numerical and string values. This was an introduction, so here are some more resources to expand your learning!

### Resources:

- Mode Analytics SQL: https://mode.com/sql-tutorial/

- MySQL Document Reference: https://dev.mysql.com/doc/refman/8.0/en/window-functions-usage.html

- BigQuery Document Reference for navigation (string and numerical value aggregation): https://cloud.google.com/bigquery/docs/reference/standard-sql/navigation_functions

- BigQuery Document Reference for WINDOW function syntax: https://cloud.google.com/bigquery/docs/reference/standard-sql/window-function-calls#syntax 

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=caf0fd0d-699f-4f6c-9bdb-260e4f7cbaf2' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>